# Inicializace

In [18]:
from collections import deque                # Trida pro ukladani stavu
from __future__ import division              # Deleni realnych cisel (kvuli verzi Pythonu 2.6)

import numpy as np                           # Knihovna pro matematicke operace
import random                                # Knihovna pro nahodny vyber samplu z pameti
import sys                                   # Pro pripojeni knihovny Open AI Gym
sys.path.append('/home/xbucha02/libraries')  # Pripojeni knihovny Open AI Gym
import gym                                   # Knihovna Open AI Gym
#from gym import wrappers                    # Nahravani zaznamu
env = gym.make('CartPole-v0')             # Konkretni hra z Open AI Gym
actionCount = env.action_space.n             # Pocet vstupu do prostredi
stateSize = env.observation_space.shape[0]   # Pocet vystupu z prostredi
#env = wrappers.Monitor(env, '/home/lachubcz/tmp/cartpole-experiment-1', force=True) #Nahravani zaznamu

gpuMemoryUsage=1                            # Vyuziti pameti GPU
import tensorflow as tf                     # Knihovna TensorFlow
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = gpuMemoryUsage
set_session(tf.Session(config=config))

# Parametry
observetime = 200                          # Delka pozorovani
episodes = 2000                            # Pocet epizod
games = 100                                # Pocet her
#trainingAfterSucces = 100                  # Pocet trenovani na uspesnem datasetu
scores = []                                # Pole pro ulozeni vysledku na analyzu
episodesList = []                          # Pole pro ulozeni cisel epizod na analyzu
bestScore = 0                              # Promenna pro ukladani nejlepsiho prubezneho vysledku

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


# Agent

In [19]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras import regularizers
from keras import optimizers
from keras import losses
from keras import metrics
import copy

class Agent:
    def __init__(self, env):
        self.startEpsilon = 1                           # Pravdepodobnost konani nahodneho tahu na zacatku
        self.endEpsilon = 0.005                          # Pravdepodobnost konani nahodneho tahu na konci
        self.currentEpsilon = self.startEpsilon         # Soucasna pravdepodobnost konani nahodneho tahu
        self.epsilonDiminution = (self.startEpsilon - self.endEpsilon)/(episodes * 2)                  # Hodnota snizovani epsilonu
        self.gamma = 0.95                               # Discount faktor
        self.minibatchSize = 32                         # Velikost minibatche
        self.actionCount = env.action_space.n           # Pocet vstupu do prostredi
        self.stateSize = env.observation_space.shape[0] # Pocet vystupu z prostredi
        self.learningRate = 0.001                       # Learning rate
        self.fractionUpdate = 0.125
        self.memorySize = 1000                          # Velikost Replay memory
        self.primaryMemory = deque(maxlen=self.memorySize)
        self.secondaryMemory = deque(maxlen=self.memorySize)
        
        self.net = self.getNN(env)
        self.netTarget = self.getNN(env)
        self.updateTargetNet()

    def getNN(self, env):
        net = Sequential()
        net.add(Dense(32, activation="relu", input_shape=(self.stateSize,)))
        #net.add(Flatten())
        net.add(Dense(16, activation="relu"))
        
        net.add(Dense(self.actionCount, activation="linear"))

        net.summary()

        net.compile(loss=losses.mean_squared_error, optimizer=optimizers.Adam(lr=self.learningRate), metrics=['accuracy'])

        return net
    
    def updateTargetNet(self):
        self.netTarget.set_weights(self.net.get_weights())
    
    def updateTargetNetPartially(self):
        weights = self.net.get_weights()
        weightsTarget = self.netTarget.get_weights()
        
        for i in range(len(weightsTarget)):
            weightsTarget[i] = weights[i] * self.fractionUpdate + weightsTarget[i] * (1 - self.fractionUpdate)
            
        self.netTarget.set_weights(weightsTarget)
        
    def rememberPrimMem(self, state, action, reward, nextState, done):
        self.primaryMemory.append((state, action, reward, nextState, done))
    
    def rememberSecMem(self, state, action, reward, nextState, done):
        self.secondaryMemory.append((state, action, reward, nextState, done))
        
    def clearSecMem(self):
        self.secondaryMemory.clear()
        
    def epsilonActulization(self):
        if self.currentEpsilon > self.endEpsilon:
            if (self.currentEpsilon - self.epsilonDiminution) > self.endEpsilon:
                self.currentEpsilon = self.currentEpsilon - self.epsilonDiminution
            else:
                self.currentEpsilon = self.endEpsilon
    
    def getActionWE(self, state):
        if np.random.rand() <= self.currentEpsilon:
            return np.random.randint(0, self.actionCount, size=1)[0]
        else:
            Q = self.net.predict(state)
            return np.argmax(Q)

    def getAction(self, state):
        Q = self.net.predict(state)
        return np.argmax(Q)
    
    def resetSecMem(self):
        self.secondaryMemory = deque(maxlen=self.memorySize)
        
    def resetEpsilon(self):
        self.currentEpsilon = self.startEpsilon
    
    def trainDQN(self):
        if len(self.primaryMemory) >= self.minibatchSize:
            minibatch = random.sample(self.primaryMemory, self.minibatchSize) #z D vybere pocet mb_size samplu
        else:
            return
        
        #states = np.empty(self.minibatchSize)
        #print("{}" .format(states))
        #target_fs = np.empty(self.minibatchSize)
        state = np.vstack([i[0] for i in minibatch])
        action = [i[1] for i in minibatch]
        reward = [i[2] for i in minibatch]
        nextState = np.vstack([i[3] for i in minibatch])
        done = [i[4] for i in minibatch]
        

        target_fs = []
        
        for i in range(0, self.minibatchSize):
            #state = minibatch[i][0]
            #action = minibatch[i][1]
            #reward = minibatch[i][2]
            #nextState = minibatch[i][3]
            #done = minibatch[i][4]

            target_f = self.net.predict(state[i])

            if done:
                target_f[0][action[i]] = reward[i]
            else:
                aNet = self.net.predict(nextState[i])[0]

                target_f[0][action[i]] = reward + self.gamma * np.max(aNet)
            target_fs.append(target_f)
            
        self.net.fit(state, target_fs, epochs=1, verbose=0)
                
    def trainDDQN(self):
        if len(self.primaryMemory) >= self.minibatchSize:
            minibatch = random.sample(self.primaryMemory, self.minibatchSize) #z D vybere pocet mb_size samplu
        else:
            return

        state = np.vstack([i[0] for i in minibatch])
        action = [i[1] for i in minibatch]
        reward = [i[2] for i in minibatch]
        nextState = np.vstack([i[3] for i in minibatch])
        done = [i[4] for i in minibatch]

        target_fs = np.zeros((self.minibatchSize, self.actionCount))

        for i in range(0, self.minibatchSize):

            target_f = self.net.predict(np.array([state[i]]))

            if minibatch[i]:
                target_f[0][action[i]] = reward[i]
            else:
                aNet = self.net.predict(nextState[i])[0]
                tNet = self.netTarget.predict(nextState[i])[0]
                target_f[0][action] = reward[i] + self.gamma * tNet[np.argmax(aNet)]

            target_fs[i][0] = target_f[0][0]
            target_fs[i][1] = target_f[0][1]

        self.net.fit(state, target_fs, batch_size = self.minibatchSize, verbose = 0, epochs = 32)
    
    def loadNN(self, name):
        self.net.load_weights(name)

    def saveNN(self, name):
        self.net.save_weights(name)

# Algoritmus

In [20]:
from tqdm import tnrange, tqdm_notebook
from time import sleep
agent = Agent(env) #vytvoreni agenta
    
#agent.loadNN("./DDQN-MountainCar-v0.h5") #nacteni NN
#agent.updateTargetNet() #nacteni NN do netTarget

for eps in tnrange(episodes, desc='episodes'):
    state = env.reset() #resetovani prostredi
    #state = np.reshape(state, [1, stateSize]) #formatovani
    #state = np.reshape(state, (1, stateSize))
    #state1 = copy.copy(state)
    #state2 = copy.copy(state)
    #state3 = copy.copy(state)
    
    #state = np.concatenate((state, state1))
    #state = np.concatenate((state, state2))
    #state = np.concatenate((state, state3))

    #state = np.expand_dims(state, axis=0)
    #print("state: {}".format(state))
     #aktualizace epsilon
    
    #env.render()

    for t in range(observetime):
        
        state = np.reshape(state, (1, stateSize))
        action = agent.getActionWE(state) #ziskani akce

        nextState, reward, done, info = env.step(action) #provedeni akce
        
        #nextState = np.reshape(newState, [1, stateSize]) #formatovani
        #nextState = np.reshape(state, (1, stateSize))
        
        #temp = copy.copy(nextState)
        
        #nextState = np.concatenate((nextState, state1))
        #nextState = np.concatenate((nextState, state2))
        #nextState = np.concatenate((nextState, state3))
        
        #state3 = copy.copy(state2)
        #state2 = copy.copy(state1)
        #state1 = copy.copy(temp)
        
        #nextState = np.expand_dims(nextState, axis=0)
        #print ("{}".format(nextState))
        #print("State: {}next_state:{}" .format(state, nextState))
        agent.rememberPrimMem(state, action, reward, nextState, done) #ulozeni do primarni pameti
        #agent.rememberPrimMem(state, action, reward, nextState, done) #ulozeni do sekundarni pameti
        
        agent.trainDDQN() #trenovani na primarni pameti
        #print("state: {}".format(nextState))
        #print("State: {}" .format(state))
        state = nextState #zmena stavu
        
        agent.updateTargetNet() #aktualizace target site
        if eps % 50 == 0:
            agent.saveNN("./DDQN-CartPole-v0.h5") #ulozeni site
        agent.epsilonActulization()
        
        if done: #konec epizody
            print("Episode: {}/{}, epsilon: {:.2}, score: {}".format(eps, episodes, agent.currentEpsilon, t))
            #agent.trainDDQN()
            scores.append(t) #ulozeni aktualniho skore
            episodesList.append(eps) #ulozeni aktualniho cisla epizody
            
            
            #if t > bestScore: #bylo dosazeno njelepsiho skore
            #    bestScore = t #nove nejlepsi skore
            #    for i in range(trainingAfterSucces): #pocet trenovani
            #        agent.trainDDQN(2) #trenovani na sekundarni pameti
            #agent.saveNN("./DDQN-CartPole-v0-{}.h5" .format(eps)) #ulozeni site
            #    #agent.resetEpsilon() #resetovani epsilonu
            
            #agent.resetSecMem() #vycisteni sekundarni pameti
            
            break
    #sleep(0.01)
            
agent.saveNN("./DDQN-CartPole-v0.h5") #ulozeni site

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_31 (Dense)             (None, 32)                160       
_________________________________________________________________
dense_32 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_33 (Dense)             (None, 2)                 34        
Total params: 722
Trainable params: 722
Non-trainable params: 0
_________________________________________________________________
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_34 (Dense)             (None, 32)                160       
_________________________________________________________________
dense_35 (Dense)             (None, 16)                528       
_________________________________________________________________
dense_36 (De

A Jupyter Widget

Episode: 0/2000, epsilon: 0.99, score: 33
Episode: 1/2000, epsilon: 0.98, score: 29
Episode: 2/2000, epsilon: 0.98, score: 23
Episode: 3/2000, epsilon: 0.97, score: 16
Episode: 4/2000, epsilon: 0.97, score: 19
Episode: 5/2000, epsilon: 0.96, score: 22



Exception in thread Thread-8:
Traceback (most recent call last):
  File "C:\Users\LachubCz_NTB\Anaconda3\envs\tensorflow\lib\threading.py", line 914, in _bootstrap_inner
    self.run()
  File "C:\Users\LachubCz_NTB\Anaconda3\envs\tensorflow\lib\site-packages\tqdm\_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "C:\Users\LachubCz_NTB\Anaconda3\envs\tensorflow\lib\_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration



Episode: 6/2000, epsilon: 0.95, score: 42
Episode: 7/2000, epsilon: 0.95, score: 13
Episode: 8/2000, epsilon: 0.95, score: 12
Episode: 9/2000, epsilon: 0.94, score: 21
Episode: 10/2000, epsilon: 0.93, score: 23
Episode: 11/2000, epsilon: 0.92, score: 55
Episode: 12/2000, epsilon: 0.92, score: 11
Episode: 13/2000, epsilon: 0.91, score: 19
Episode: 14/2000, epsilon: 0.91, score: 15
Episode: 15/2000, epsilon: 0.9, score: 13
Episode: 16/2000, epsilon: 0.9, score: 11
Episode: 17/2000, epsilon: 0.9, score: 11
Episode: 18/2000, epsilon: 0.9, score: 10
Episode: 19/2000, epsilon: 0.89, score: 10
Episode: 20/2000, epsilon: 0.89, score: 21
Episode: 21/2000, epsilon: 0.89, score: 8
Episode: 22/2000, epsilon: 0.88, score: 23
Episode: 23/2000, epsilon: 0.88, score: 13
Episode: 24/2000, epsilon: 0.87, score: 36
Episode: 25/2000, epsilon: 0.86, score: 13
Episode: 26/2000, epsilon: 0.86, score: 12
Episode: 27/2000, epsilon: 0.86, score: 12
Episode: 28/2000, epsilon: 0.85, score: 12
Episode: 29/2000, ep

KeyboardInterrupt: 

# Výsledky

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt
import copy

def analysis(scores, episodesList):
    score1 = copy.copy(scores)
    score2 = copy.copy(scores)
    score3 = copy.copy(scores)

    for i in range (len(scores)):
        if i > 1 and i < (len(scores)-2):
            score1[i] = (scores[i - 2] + scores[i - 1] + scores[i] + scores[i + 1] + scores[i + 2])/5

    for i in range (len(scores)):
        if i > 4 and i < (len(scores)-5):
            score2[i] += scores[i - 5] + scores[i - 4] + scores[i - 3] + scores[i - 2] + scores[i - 1]
            score2[i] += scores[i + 5] + scores[i + 4] + scores[i + 3] + scores[i + 2] + scores[i + 1]
            score2[i] = score2[i]/11

    for i in range (len(scores)):
        if i > 49 and i < (len(scores) - 50):
            for e in range (1,50):
                score3[i] += scores[i - e] + scores[i + e] 
            score3[i] = score3[i]/101      

    plt.plot(episodesList, scores, 'ro')
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Vysledky")
    plt.show()

    plt.plot(scores)
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Funkce interpolace vysledku")
    plt.show()

    plt.plot(score1)
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Funkce interpolace vysledku (filtr - prumer 5-ti prvku)")
    plt.show()

    plt.plot(score2)
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Funkce interpolace vysledku  (filtr - prumer 11-cti prvku)")
    plt.show()

    plt.plot(score3)
    plt.ylabel("Score")
    plt.xlabel("Episodes")
    plt.title("Funkce interpolace vysledku  (filtr - prumer 101 prvku)")
    plt.show()
    
analysis(scores, episodesList)

# Hra

In [ ]:
agent.loadNN("./DDQN-CartPole-v0.h5")
observation = env.reset()
scores = []                                # Vycisteni pole pro ulozeni vysledku na analyzu
episodesList = []                          # Vycisteni pole pro ulozeni cisel epizod na analyzu

for g in range (games):
    state = env.reset()
    state = np.reshape(state, [1, stateSize])
    
    state1 = copy.copy(state)
    state2 = copy.copy(state)
    state3 = copy.copy(state)
    
    state = np.concatenate((state, state1))
    state = np.concatenate((state, state2))
    state = np.concatenate((state, state3))

    state = np.expand_dims(state, axis=0)
    
    totalReward = 0
    done = False
    
    while not done:
        #env.render()
        
        action = agent.getAction(state)
        newState, reward, done, info = env.step(action)

        nextState = np.reshape(newState, [1, stateSize])

        temp = copy.copy(nextState)
        
        nextState = np.concatenate((nextState, state1))
        nextState = np.concatenate((nextState, state2))
        nextState = np.concatenate((nextState, state3))
        
        state3 = copy.copy(state2)
        state2 = copy.copy(state1)
        state1 = copy.copy(temp)
        
        nextState = np.expand_dims(nextState, axis=0)
        
        state = nextState

        totalReward += reward
        
    scores.append(totalReward) #ulozeni aktualniho skore
    episodesList.append(g) #ulozeni aktualniho cisla epizody

    print('Game {}/{}, score: {}'.format(g, games, totalReward))

analysis(scores, episodesList)